In [1]:
import pandas as pd 
from pandas import Series, DataFrame
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient
import numpy as np

In [2]:
df_raw = pd.read_csv("NYPD_Arrest_Data__Year_to_Date__20240229.csv")
print(df_raw.info())
print(df_raw.shape)
print(df_raw.columns)
df_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226872 entries, 0 to 226871
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ARREST_KEY                226872 non-null  int64  
 1   ARREST_DATE               226872 non-null  object 
 2   PD_CD                     226870 non-null  float64
 3   PD_DESC                   226872 non-null  object 
 4   KY_CD                     226855 non-null  float64
 5   OFNS_DESC                 226872 non-null  object 
 6   LAW_CODE                  226872 non-null  object 
 7   LAW_CAT_CD                225273 non-null  object 
 8   ARREST_BORO               226872 non-null  object 
 9   ARREST_PRECINCT           226872 non-null  int64  
 10  JURISDICTION_CODE         226872 non-null  int64  
 11  AGE_GROUP                 226872 non-null  object 
 12  PERP_SEX                  226872 non-null  object 
 13  PERP_RACE                 226872 non-null  o

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,New Georeferenced Column
0,261265483,01/03/2023,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1600500,F,B,49,0,18-24,M,BLACK,1027430,251104,40.855793,-73.843908,POINT (-73.843908 40.855793)
1,261271301,01/03/2023,105.0,STRANGULATION 1ST,106.0,FELONY ASSAULT,PL 1211200,F,S,120,0,25-44,M,WHITE,962808,174275,40.644996,-74.077263,POINT (-74.077263 40.644996)
2,261336449,01/04/2023,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1601001,F,K,61,0,<18,M,BLACK,995118,155708,40.594054,-73.960866,POINT (-73.960866 40.594054)
3,261328047,01/04/2023,105.0,STRANGULATION 1ST,106.0,FELONY ASSAULT,PL 1211200,F,Q,114,0,18-24,M,BLACK,1007694,219656,40.769552,-73.915361,POINT (-73.915361 40.769552)
4,261417496,01/05/2023,244.0,"BURGLARY,UNCLASSIFIED,UNKNOWN",107.0,BURGLARY,PL 1402000,F,B,44,0,25-44,F,BLACK,1007174,239542,40.824135,-73.917170,POINT (-73.91717 40.824135)


In [3]:
#cleaning 
#removing'X_COORD_CD', 'Y_COORD_CD','New Georeferenced Column', 'PD_CD','KY_CD','JURISDICTION_CODE'

df_cleaned = df_raw.drop(columns= ['X_COORD_CD', 'Y_COORD_CD','New Georeferenced Column', 'PD_CD','KY_CD','JURISDICTION_CODE'])

#dropping NA values
df_cleaned.dropna()

df_cleaned.head()

,ARREST_KEY,ARREST_DATE,PD_DESC,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,AGE_GROUP,PERP_SEX,PERP_RACE,Latitude,Longitude
0,261265483,01/03/2023,"ROBBERY,OPEN AREA UNCLASSIFIED",ROBBERY,PL 1600500,F,B,49,18-24,M,BLACK,40.855793,-73.843908
1,261271301,01/03/2023,STRANGULATION 1ST,FELONY ASSAULT,PL 1211200,F,S,120,25-44,M,WHITE,40.644996,-74.077263
2,261336449,01/04/2023,"ROBBERY,OPEN AREA UNCLASSIFIED",ROBBERY,PL 1601001,F,K,61,<18,M,BLACK,40.594054,-73.960866
3,261328047,01/04/2023,STRANGULATION 1ST,FELONY ASSAULT,PL 1211200,F,Q,114,18-24,M,BLACK,40.769552,-73.915361
4,261417496,01/05/2023,"BURGLARY,UNCLASSIFIED,UNKNOWN",BURGLARY,PL 1402000,F,B,44,25-44,F,BLACK,40.824135,-73.917170


In [4]:
# creating dim_location

#cleaning
boro_mapping = {
    'B': 'Bronx',
    'S': 'Staten Island',
    'K': 'Brooklyn',
    'M': 'Manhattan',
    'Q': 'Queens'
}

df_cleaned['ARREST_BORO'] = df_cleaned['ARREST_BORO'].map(boro_mapping)


location_columns = ['Longitude', 'Latitude', 'ARREST_PRECINCT', 'ARREST_BORO']
dim_location = df_cleaned[location_columns].drop_duplicates().reset_index(drop=True)

# Assigning a unique identifier to each Location
dim_location['location_id'] = dim_location.index + 1  # Starts IDs at 1

# Create a mapping dictionary 
location_mapping = dim_location.set_index(['Longitude', 'Latitude', 'ARREST_PRECINCT', 'ARREST_BORO'])['location_id'].to_dict()


df_cleaned['location_id'] = df_cleaned.set_index(['Longitude', 'Latitude', 'ARREST_PRECINCT', 'ARREST_BORO']).index.map(location_mapping.get)


print(df_cleaned[['ARREST_PRECINCT', 'ARREST_BORO', 'Longitude', 'Latitude', 'location_id']].head())

   ARREST_PRECINCT    ARREST_BORO  Longitude   Latitude  location_id
0               49          Bronx -73.843908  40.855793            1
1              120  Staten Island -74.077263  40.644996            2
2               61       Brooklyn -73.960866  40.594054            3
3              114         Queens -73.915361  40.769552            4
4               44          Bronx -73.917170  40.824135            5


In [5]:
# creating dim_offense

# Filtering out
df_cleaned = df_cleaned[~df_cleaned['LAW_CAT_CD'].isin(['null', 'nan', '9', 'I'])]

df_cleaned = df_cleaned.dropna(subset=['LAW_CAT_CD'])

# Apply the mapping to convert single characters to full words
law_cat_mapping = {
    'F': 'Felony',
    'M': 'Misdemeanor',
    'V': 'Violation'
}

# Update 'LAW_CAT_CD' to the full category names
df_cleaned['LAW_CAT_CD'] = df_cleaned['LAW_CAT_CD'].map(law_cat_mapping)


offense_columns = ['OFNS_DESC', 'PD_DESC', 'LAW_CODE', 'LAW_CAT_CD']
dim_offense = df_cleaned[offense_columns].drop_duplicates().reset_index(drop=True)

# Assigning a unique identifier to each offense
dim_offense['offense_id'] = dim_offense.index + 1  # Start IDs at 1


offense_mapping = dim_offense.set_index(['OFNS_DESC', 'PD_DESC', 'LAW_CODE', 'LAW_CAT_CD'])['offense_id'].to_dict()


df_cleaned['offense_id'] = df_cleaned.set_index(['OFNS_DESC', 'PD_DESC', 'LAW_CODE', 'LAW_CAT_CD']).index.map(offense_mapping)


print(dim_offense.head())



        OFNS_DESC                         PD_DESC    LAW_CODE LAW_CAT_CD  \
0         ROBBERY  ROBBERY,OPEN AREA UNCLASSIFIED  PL 1600500     Felony   
1  FELONY ASSAULT               STRANGULATION 1ST  PL 1211200     Felony   
2         ROBBERY  ROBBERY,OPEN AREA UNCLASSIFIED  PL 1601001     Felony   
3        BURGLARY   BURGLARY,UNCLASSIFIED,UNKNOWN  PL 1402000     Felony   
4  FELONY ASSAULT        ASSAULT 2,1,UNCLASSIFIED  PL 1200502     Felony   

   offense_id  
0           1  
1           2  
2           3  
3           4  
4           5  


In [6]:
#creating dim_perp

sex_mapping = {
    'M': 'Male',
    'F': 'Female',
    'U': 'Unknown'
}
df_cleaned['PERP_SEX'] = df_cleaned['PERP_SEX'].map(sex_mapping)


df_cleaned = df_cleaned.dropna(subset=['AGE_GROUP', 'PERP_SEX', 'PERP_RACE'])


demo_columns = ['AGE_GROUP', 'PERP_SEX', 'PERP_RACE']
dim_demo_perp = df_cleaned[demo_columns].drop_duplicates().reset_index(drop=True)

# Assigning a unique identifier to each demographic profile
dim_demo_perp['demo_perp_id'] = dim_demo_perp.index + 1  # Start IDs at 1

# Create the mapping dictionary
demo_mapping = dim_demo_perp.set_index(['AGE_GROUP', 'PERP_SEX', 'PERP_RACE'])['demo_perp_id'].to_dict()


df_cleaned['demo_perp_id'] = df_cleaned.set_index(['AGE_GROUP', 'PERP_SEX', 'PERP_RACE']).index.map(demo_mapping.get)


df_cleaned.reset_index(drop=True, inplace=True)


print(df_cleaned[['AGE_GROUP', 'PERP_SEX', 'PERP_RACE', 'demo_perp_id']].head())


  AGE_GROUP PERP_SEX PERP_RACE  demo_perp_id
0     18-24     Male     BLACK             1
1     25-44     Male     WHITE             2
2       <18     Male     BLACK             3
3     18-24     Male     BLACK             1
4     25-44   Female     BLACK             4


In [7]:
#creating dim_date

df_cleaned['ARREST_DATE'] = pd.to_datetime(df_cleaned['ARREST_DATE'])

#cleaning
dim_date = df_cleaned[['ARREST_DATE']].drop_duplicates().reset_index(drop=True)

# Extract different parts of the date into separate columns
dim_date['year'] = dim_date['ARREST_DATE'].dt.year
dim_date['monthNum'] = dim_date['ARREST_DATE'].dt.month
dim_date['weekOfMonth'] = dim_date['ARREST_DATE'].apply(lambda x: x.day // 7 + 1)
dim_date['weekOfYear'] = dim_date['ARREST_DATE'].dt.isocalendar().week
dim_date['dayNum'] = dim_date['ARREST_DATE'].dt.day
dim_date['dayName'] = dim_date['ARREST_DATE'].dt.day_name()
dim_date['monthName'] = dim_date['ARREST_DATE'].dt.month_name()

# Assign a unique identifier to each date
dim_date['date_id'] = dim_date.index + 1  # Starts IDs at 1

# Reorganizing columns
dim_date = dim_date[['date_id', 'ARREST_DATE','year', 'monthNum', 'weekOfMonth', 
                     'weekOfYear', 'dayNum', 'dayName', 'monthName']]


print(dim_date.head())

date_mapping = dim_date.set_index('ARREST_DATE')['date_id'].to_dict()


df_cleaned['date_id'] = df_cleaned['ARREST_DATE'].map(date_mapping)

   date_id ARREST_DATE  year  monthNum  weekOfMonth  weekOfYear  dayNum  \
0        1  2023-01-03  2023         1            1           1       3   
1        2  2023-01-04  2023         1            1           1       4   
2        3  2023-01-05  2023         1            1           1       5   
3        4  2023-01-08  2023         1            2           1       8   
4        5  2023-01-09  2023         1            2           2       9   

     dayName monthName  
0    Tuesday   January  
1  Wednesday   January  
2   Thursday   January  
3     Sunday   January  
4     Monday   January  


In [8]:
# Create a new DataFrame for the fact table
facts_police_activity = pd.DataFrame()

# Add the foreign keys from the dimension tables
facts_police_activity['location_id'] = df_cleaned['location_id']
facts_police_activity['offense_id'] = df_cleaned['offense_id']
facts_police_activity['date_id'] = df_cleaned['date_id']
facts_police_activity['demo_perp_id'] = df_cleaned['demo_perp_id']


facts_police_activity['arrest_key'] = df_cleaned['ARREST_KEY']



facts_police_activity['ARREST_BORO'] = df_cleaned['ARREST_BORO']


freq_arrest_by_boro = df_cleaned.groupby('ARREST_BORO').size().reset_index(name='freq_arrest_by_boro')


facts_police_activity = facts_police_activity.merge(freq_arrest_by_boro, on='ARREST_BORO', how='left')


facts_police_activity['incident_count'] = 1

# Assign a unique identifier for each activity
facts_police_activity['policies_activity_id'] = range(1, len(facts_police_activity) + 1)

facts_police_activity.drop('ARREST_BORO', axis=1, inplace=True)


print(facts_police_activity.head())


   location_id  offense_id  date_id  demo_perp_id  arrest_key  \
0            1           1        1             1   261265483   
1            2           2        1             2   261271301   
2            3           3        2             3   261336449   
3            4           2        2             1   261328047   
4            5           4        3             4   261417496   

   freq_arrest_by_boro  incident_count  policies_activity_id  
0                52870               1                     1  
1                10001               1                     2  
2                62094               1                     3  
3                47501               1                     4  
4                52870               1                     5  


In [25]:
#merging all into new dataframe 
df_transformed = facts_police_activity.copy()

# Merge with the dim_location table on location_id
df_transformed = df_transformed.merge(dim_location, on='location_id', how='left')

# Merge with the dim_offense table on offense_id
df_transformed = df_transformed.merge(dim_offense, on='offense_id', how='left')

# Merge with the dim_demo_perp table on demo_perp_id
df_transformed = df_transformed.merge(dim_demo_perp, on='demo_perp_id', how='left')

# Merge with the dim_date table on date_id
df_transformed = df_transformed.merge(dim_date, on='date_id', how='left')

# reorganized
column_order = [
    'policies_activity_id', 'location_id', 'offense_id', 'date_id', 'demo_perp_id', 
    'arrest_key', 'freq_arrest_by_boro', 'incident_count', 'ARREST_PRECINCT', 'LAW_CAT_CD',
    'ARREST_BORO','AGE_GROUP', 'PERP_SEX', 'PERP_RACE', 'ARREST_DATE', 'year', 'monthNum', 
    'weekOfMonth', 'weekOfYear', 'dayNum', 'dayName', 'monthName', 'Longitude','Latitude',
    'PD_DESC','OFNS_DESC'
]


# Reindex 
df_transformed = df_transformed[column_order]

print(df_transformed.head())


   policies_activity_id  location_id  offense_id  date_id  demo_perp_id  \
0                     1            1           1        1             1   
1                     2            2           2        1             2   
2                     3            3           3        2             3   
3                     4            4           2        2             1   
4                     5            5           4        3             4   

   arrest_key  freq_arrest_by_boro  incident_count  ARREST_PRECINCT  \
0   261265483                52870               1               49   
1   261271301                10001               1              120   
2   261336449                62094               1               61   
3   261328047                47501               1              114   
4   261417496                52870               1               44   

  LAW_CAT_CD  ... monthNum weekOfMonth weekOfYear dayNum    dayName  \
0     Felony  ...        1           1          1  

In [26]:
df_transformed.info()
df_transformed.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224416 entries, 0 to 224415
Data columns (total 26 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   policies_activity_id  224416 non-null  int64         
 1   location_id           224416 non-null  int64         
 2   offense_id            224416 non-null  int64         
 3   date_id               224416 non-null  int64         
 4   demo_perp_id          224416 non-null  int64         
 5   arrest_key            224416 non-null  int64         
 6   freq_arrest_by_boro   224416 non-null  int64         
 7   incident_count        224416 non-null  int64         
 8   ARREST_PRECINCT       224416 non-null  int64         
 9   LAW_CAT_CD            224414 non-null  object        
 10  ARREST_BORO           224416 non-null  object        
 11  AGE_GROUP             224416 non-null  object        
 12  PERP_SEX              224416 non-null  object        
 13 

,policies_activity_id,location_id,offense_id,date_id,demo_perp_id,arrest_key,freq_arrest_by_boro,incident_count,ARREST_PRECINCT,LAW_CAT_CD,...,monthNum,weekOfMonth,weekOfYear,dayNum,dayName,monthName,Longitude,Latitude,PD_DESC,OFNS_DESC
0,1,1,1,1,1,261265483,52870,1,49,Felony,...,1,1,1,3,Tuesday,January,-73.843908,40.855793,"ROBBERY,OPEN AREA UNCLASSIFIED",ROBBERY
1,2,2,2,1,2,261271301,10001,1,120,Felony,...,1,1,1,3,Tuesday,January,-74.077263,40.644996,STRANGULATION 1ST,FELONY ASSAULT
2,3,3,3,2,3,261336449,62094,1,61,Felony,...,1,1,1,4,Wednesday,January,-73.960866,40.594054,"ROBBERY,OPEN AREA UNCLASSIFIED",ROBBERY
3,4,4,2,2,1,261328047,47501,1,114,Felony,...,1,1,1,4,Wednesday,January,-73.915361,40.769552,STRANGULATION 1ST,FELONY ASSAULT
4,5,5,4,3,4,261417496,52870,1,44,Felony,...,1,1,1,5,Thursday,January,-73.917170,40.824135,"BURGLARY,UNCLASSIFIED,UNKNOWN",BURGLARY


In [28]:
import os

CONNECTION_STRING_AZURE_STORAGE = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

if not CONNECTION_STRING_AZURE_STORAGE:
    raise ValueError("No Azure Storage connection string found")

CONTAINER_AZURE = 'transfromed-nypd-arrest'
blob_name = "nypd-arrest-transfrom.csv"

# Convert DataFrame to CSV and get the string
output = StringIO()
df_transformed.to_csv(output, index=False)
data = output.getvalue()
output.close()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(data, overwrite=True)

# Print confirmation message
print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")

Uploaded nypd-arrest-transfrom.csv to Azure Blob Storage in container transfromed-nypd-arrest.
